In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark=SparkSession.builder.getOrCreate()


# to display noteboke cell with harizontal scroll bar 
from IPython.display import display_html




Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/23 23:56:05 WARN Utils: Your hostname, ahmed-refat-VirtualBox, resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/10/23 23:56:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/23 23:56:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
from pyspark.sql import functions as fn
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, LongType

spark = SparkSession.builder.appName("Dataset Example").getOrCreate()


data = [
    (1, "Ahmed", "Refat", "https://ahmedrefat.com", "2023-05-01", 1200, "Summer Campaign"),
    (2, "Mona", None, "https://monakhaled.net", None, 900, "Spring Campaign"),
    (3, None, "Sami", None, "2023-07-20", None, "Back to School"),
    (4, "Sara", "Mostafa", "https://saramostafa.io", "2023-08-12", 3200, None),
    (5, "Hany", "Ali", None, "2023-09-01", 1800, "Winter Launch"),
    (6, None, "Fathy", "https://norafathy.me", "2023-09-20", 2600, "Autumn Ads")
]
schema = StructType([
    StructField("id", IntegerType(), True), # يعني عادي تنفع تبقا ب نل او تتكرر
    StructField("first", StringType(), True),
    StructField("last", StringType(), True),
    StructField("url", StringType(), True),
    StructField("published", StringType(), True),   # ممكن نخليها DateType كمان
    StructField("hits", LongType(), True),
    StructField("campaigns", StringType(), True)
])

df = spark.createDataFrame(data, schema=schema)
df.show(truncate=False)
#df.printSchema()



+---+-----+-------+----------------------+----------+----+---------------+
|id |first|last   |url                   |published |hits|campaigns      |
+---+-----+-------+----------------------+----------+----+---------------+
|1  |Ahmed|Refat  |https://ahmedrefat.com|2023-05-01|1200|Summer Campaign|
|2  |Mona |NULL   |https://monakhaled.net|NULL      |900 |Spring Campaign|
|3  |NULL |Sami   |NULL                  |2023-07-20|NULL|Back to School |
|4  |Sara |Mostafa|https://saramostafa.io|2023-08-12|3200|NULL           |
|5  |Hany |Ali    |NULL                  |2023-09-01|1800|Winter Launch  |
|6  |NULL |Fathy  |https://norafathy.me  |2023-09-20|2600|Autumn Ads     |
+---+-----+-------+----------------------+----------+----+---------------+



احنا عملنا عمليه بناء ل الداتا فريم وحددنا الاسكيما بتعتها  بس دلوقتي عندنا في قيم مش مضبوطه وفي قيم فاضيه 
هنحاول اننا نعالج كل الموضوع ده بكل الطرق والامكانيات المتاحه 

In [15]:
# كود معرفه عدد النل في كل عمود

df.select([fn.count(fn.when(fn.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

# معني الكود الي جي انك تحذف اي صف في قيم نل في اي مكان 
df.dropna().show()


+---+-----+----+---+---------+----+---------+
| id|first|last|url|published|hits|campaigns|
+---+-----+----+---+---------+----+---------+
|  0|    2|   1|  2|        1|   1|        1|
+---+-----+----+---+---------+----+---------+

+---+-----+-----+--------------------+----------+----+---------------+
| id|first| last|                 url| published|hits|      campaigns|
+---+-----+-----+--------------------+----------+----+---------------+
|  1|Ahmed|Refat|https://ahmedrefa...|2023-05-01|1200|Summer Campaign|
+---+-----+-----+--------------------+----------+----+---------------+



In [ ]:
# الكود ده هيحذف كل القيم الفاضيه في اعمده معينه فقط
df.dropna(subset=["first","last"]).show()


# احذف الصفوف اللي فيها أي Null (ده الافتراضي)
df.dropna(how='any').show()

# احذف الصفوف اللي كلها Null
df.dropna(how='all').show()

# احذف الصفوف اللي فيها Null في 2 أو أكتر من الأعمدة
df.dropna(thresh=5).show()



+---+-----+-------+--------------------+----------+----+---------------+
| id|first|   last|                 url| published|hits|      campaigns|
+---+-----+-------+--------------------+----------+----+---------------+
|  1|Ahmed|  Refat|https://ahmedrefa...|2023-05-01|1200|Summer Campaign|
|  4| Sara|Mostafa|https://saramosta...|2023-08-12|3200|           NULL|
|  5| Hany|    Ali|                NULL|2023-09-01|1800|  Winter Launch|
+---+-----+-------+--------------------+----------+----+---------------+



In [18]:
# هنبدء عمليه معالجه القيم الي مهيش موجوده او ب نان

# اول حاجه هنعوض مكان كل القيم ب قيمه ثابته الي هي unknown
df.fillna("unknown").show


# تحديد هنكتب ايه في كل قميه علي حسب العمود 
df.fillna({"first":"first",
           "last" :"last",
           "hits" :0
           }).show()

+---+-----+-------+--------------------+----------+----+---------------+
| id|first|   last|                 url| published|hits|      campaigns|
+---+-----+-------+--------------------+----------+----+---------------+
|  1|Ahmed|  Refat|https://ahmedrefa...|2023-05-01|1200|Summer Campaign|
|  2| Mona|   last|https://monakhale...|      NULL| 900|Spring Campaign|
|  3|first|   Sami|                NULL|2023-07-20|   0| Back to School|
|  4| Sara|Mostafa|https://saramosta...|2023-08-12|3200|           NULL|
|  5| Hany|    Ali|                NULL|2023-09-01|1800|  Winter Launch|
|  6|first|  Fathy|https://norafathy.me|2023-09-20|2600|     Autumn Ads|
+---+-----+-------+--------------------+----------+----+---------------+



In [23]:
# ممكن نعوض مكان قيم ارقام ب المتوسط 
df_avg = df.select(fn.avg("hits")).collect()[0][0]

df.fillna({"hits":df_avg}).show()

+---+-----+-------+--------------------+----------+----+---------------+
| id|first|   last|                 url| published|hits|      campaigns|
+---+-----+-------+--------------------+----------+----+---------------+
|  1|Ahmed|  Refat|https://ahmedrefa...|2023-05-01|1200|Summer Campaign|
|  2| Mona|   NULL|https://monakhale...|      NULL| 900|Spring Campaign|
|  3| NULL|   Sami|                NULL|2023-07-20|1940| Back to School|
|  4| Sara|Mostafa|https://saramosta...|2023-08-12|3200|           NULL|
|  5| Hany|    Ali|                NULL|2023-09-01|1800|  Winter Launch|
|  6| NULL|  Fathy|https://norafathy.me|2023-09-20|2600|     Autumn Ads|
+---+-----+-------+--------------------+----------+----+---------------+



In [ ]:
# الهدف نملي مكان url  بكلمه غير معروفه ولو كانت موجوده تتحط وهكذا في الهيتس بس هنحط قيمه النن ب 0

df = df.withColumn(
    "url",
    fn.when(fn.col("url").isNull(), "unknown.com").otherwise(fn.col("url"))
)

df = df.withColumn(
    "hits",
    fn.when(fn.col("hits").isNull(), fn.lit(0)).otherwise(fn.col("hits"))
)

df.show()
